In [11]:
!pip install nltk
!pip install pickle
!pip install xgboost
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
#from wordcloud import WordCloud
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Lasso
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

# Load your data (replace with your actual data loading)
# Assuming your data is in a CSV file named 'data.csv'
import pandas as pd
new_columns=pd.read_csv('azureml://subscriptions/512a781e-d15a-4734-adca-96ec827531cb/resourcegroups/xwang3306-rg/workspaces/DVA_PROJECT/datastores/workspaceblobstore/paths/UI/2024-11-05_152711_UTC/neiss_10p_sample_new_columns.csv')
data_10P = pd.read_csv('../xwang3306/NEISS_10P_Sample.csv')
sematic=pd.read_csv('../xwang3306/10P_sematic_bert.csv')
embedding=pd.read_csv('../xwang3306/gist_embedding_10p.csv')
#embedding_2=pd.read_csv('../xwang3306/aliba_embedding_10p.csv')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[nltk_data] Downloading pack

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [2]:
#data_10p=data.sample(frac=0.1,random_state=42).reset_index(drop=True)
#data_10p.to_csv('NEISS_10P_Sample.csv')

In [12]:
data=data_10P.merge(new_columns,how='inner',on='CPSC_Case_Number').merge(sematic,how='inner',on='CPSC_Case_Number').merge(embedding,how='inner',on='CPSC_Case_Number').reset_index(drop=True)
data.rename(columns={"sematic_distance_bert": "sematic_distance"},inplace=True)
data.head(5)

,Unnamed: 0,CPSC_Case_Number,Treatment_Date,Age,Sex,Race,Other_Race,Hispanic,Body_Part,Diagnosis,...,Narrative,Stratum,PSU,Weight,Year,Month,Day,activity_at_injury,injury_mechanism,object_involved
0,0,221032332,2022-09-24,14,1,0,0,0.0,34,71,...,14YOM REPORTS HE FELL 1 WEEK AND COMPLAINS OF ...,V,77,17.2223,2022,9,24,playing basketball,fell,basketball
1,1,181109464,2018-10-30,28,1,1,0,0.0,79,71,...,A 28YOM BENT TO PICK UP CRATE AT HOME TO ED WI...,V,25,17.5136,2018,10,30,pick up crate,bent,crate
2,2,210103105,2020-10-24,35,1,0,0,0.0,30,53,...,35YOMRIDING ON MOUNTAIN BIKE PRACTICING FELL D...,S,27,76.0369,2020,10,24,riding on mountain bike practicing,fell,mountain bike
3,3,161157997,2016-11-15,214,2,0,0,0.0,76,53,...,14 MONTH OLD FEMALE ABRASION FOR NOSE AND FORE...,S,48,85.2143,2016,11,15,coming down stairs,fell,stairs
4,4,181107411,2018-10-21,4,1,0,0,0.0,92,72,...,4YR M PLAYING WITH TOY KITCHEN APPLIANCE AND ...,C,20,4.9383,2018,10,21,playing with toy kitchen appliance,caught,toy kitchen appliance


In [13]:
import numpy as np
data['#_prod']=np.nan
data.loc[((data['Product_1']>0)), '#_prod'] = 1
data.loc[((data['Product_1']>0) & (data['Product_2']>0)), '#_prod'] = 2
data.loc[((data['Product_1']>0) & (data['Product_2']>0) & (data['Product_3']>0)), '#_prod'] = 3

In [14]:
import numpy as np
data['Disposition_recode']=np.nan
data.loc[((data['Disposition']==1)), 'Disposition_recode'] = 0
data.loc[((data['Disposition']==2)), 'Disposition_recode'] = 1
data.loc[((data['Disposition']==4)), 'Disposition_recode'] = 2
data.loc[((data['Disposition']==5)), 'Disposition_recode'] = 3
data.loc[((data['Disposition']==8)), 'Disposition_recode'] = 4
data=data[data['Disposition_recode'].notna()]

data['Disposition_recode_2']=0
data.loc[((data['Disposition_recode']>0)), 'Disposition_recode_2'] = 1

In [ ]:
data=data[(data['Body_Part']!=0) & (data['Body_Part']!=84) & (data['Body_Part']!=85) & (data['Body_Part']!=86) & (data['Body_Part']!=87)]
data['Disposition_recode'].value_counts()
data['Disposition_recode_2'].value_counts() 

In [16]:
bdpt_dict={}
bdpt_dict[0]='INTERNAL'
bdpt_dict[30]='SHOULDER'
bdpt_dict[31]='UPPERTRUNK'
bdpt_dict[32]='ELBOW'
bdpt_dict[33]='LOWERARM'
bdpt_dict[34]='WRIST'
bdpt_dict[35]='KNEE'
bdpt_dict[36]='LOWERLEG'
bdpt_dict[37]='ANKLE'
bdpt_dict[38]='PUBICREGION'
bdpt_dict[75]='HEAD'
bdpt_dict[76]='FACE'
bdpt_dict[77]='EYEBALL'
bdpt_dict[78]='UPPERTRUNK(OLD)'
bdpt_dict[79]='LOWERTRUNK'
bdpt_dict[80]='UPPERARM'
bdpt_dict[81]='UPPERLEG'
bdpt_dict[82]='HAND'
bdpt_dict[83]='FOOT'
bdpt_dict[84]='25-50% OF BODY'
bdpt_dict[85]='ALLPARTSBODY'
bdpt_dict[86]='OTHER(OLD)'
bdpt_dict[87]='NOTSTATED/UNK'
bdpt_dict[88]='MOUTH'
bdpt_dict[89]='NECK'
bdpt_dict[90]='LOWERARM(OLD)'
bdpt_dict[91]='LOWERLEG(OLD)'
bdpt_dict[92]='FINGER'
bdpt_dict[93]='TOE'
bdpt_dict[94]='EAR'


In [17]:
data['body_string']=data['Body_Part'].map(bdpt_dict)
data['Narrative_LLM']=data["activity_at_injury"].astype(str) + ' '+data["injury_mechanism"].astype(str)+ ' ' + data["object_involved"].astype(str)

In [9]:
def remove_after_dx(narrative):
  if isinstance(narrative, str):
    parts = narrative.split("DX", 1)
    if len(parts) > 1:
      return parts[0]
    else:
      return narrative  # No "DX:" found, return the original string
  else:
    return narrative  # Not a string, return as is

data['Narrative'] = data['Narrative'].apply(remove_after_dx)

In [10]:
import multiprocessing as mp
import re
sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

medical_terms = {
    "&": "and",
    "***": "",
    ">>": "clinical diagnosis",
    "@": "at",
    "abd": "abdomen",
    "af": "accidental fall",
    "afib": "atrial fibrillation",
    "aki": "acute kidney injury",
    "am": "morning",
    "ams": "altered mental status",
    "bac": "blood alcohol content",
    "bal": "blood alcohol level,",
    "biba": "brought in by ambulance",
    "c/o": "complains of",
    "chi": "closed-head injury",
    "clsd": "closed",
    "cpk": "creatine phosphokinase",
    "cva": "cerebral vascular accident",
    "dx": "clinical diagnosis",
    "ecf": "extended-care facility",
    "er": "emergency room",
    "etoh": "ethyl alcohol",
    "eval": "evaluation",
    "fd": "fall detected",
    "fx": "fracture",
    "fxs": "fractures",
    "glf": "ground level fall",
    "h/o": "history of",
    "htn": "hypertension",
    "hx": "history of",
    "inj": "injury",
    "inr": "international normalized ratio",
    "intox": "intoxication",
    "l": "left",
    "loc": "loss of consciousness",
    "lt": "left",
    "mech": "mechanical",
    "mult": "multiple",
    "n.h.": "nursing home",
    "nh": "nursing home",
    "p/w": "presents with",
    "pm": "afternoon",
    "pt": "patient",
    "pta": "prior to arrival",
    "pts": "patient's",
    "px": "physical examination", # not "procedure",
    "r": "right",
    "r/o": "rules out",
    "rt": "right",
    "s'd&f": "slipped and fell",
    "s/p": "after",
    "sah": "subarachnoid hemorrhage",
    "sdh": "acute subdural hematoma",
    "sts": "sit-to-stand",
    "t'd&f": "tripped and fell",
    "tr": "trauma",
    "uti": "urinary tract infection",
    "w/": "with",
    "w/o": "without",
    "wks": "weeks"
}

# cleanning
def clean_narrative(text):
    # lowercase everything
    text = text.lower()
    
    # unglue DX
    regex_dx = r"([ˆ\W]*(dx)[ˆ\W]*)"
    text = re.sub(regex_dx, r". dx: ", text)

    # remove age and sex identifications
    ## regex to capture age and sex (not perfect but captures almost all of the cases)
    regex_age_sex = r"(\d+)\s*?(yof|yf|yo\s*female|yo\s*f|yom|ym|yr|yo\s*male|yo\s*m)"
    age_sex_match = re.search(regex_age_sex, text)

    ## format age and sex
    if age_sex_match:
        age = age_sex_match.group(1)
        sex = age_sex_match.group(2)
        
        # probably not best practice but it works with this data
        if "f" in sex:
            #text = text.replace(age_sex_match.group(0), f"{age} years old female")
            text = text.replace(age_sex_match.group(0), f"patient")
        elif "m" in sex:
            #text = text.replace(age_sex_match.group(0), f"{age} years old male")
            text = text.replace(age_sex_match.group(0), f"patient")
    
    # translate medical terms
    for term, replacement in medical_terms.items():
        if term == "@" or term == ">>" or term == "&" or term == "***":
            pattern = fr"({re.escape(term)})"
            text = re.sub(pattern, f" {replacement} ", text) # force spaces around replacement
            
        else:
            pattern = fr"(?<!-)\b({re.escape(term)})\b(?!-)"
            text = re.sub(pattern, replacement, text)

    # user-friendly format
    sentences = sent_tokenizer.tokenize(text)
    sentences = [sent.capitalize() for sent in sentences]
    return " ".join(sentences)

text = '72 YOF SLIPPED AND FELL ON THE FLOOR THIS AM. DX: L-3, L-4 FRACTURE, RIGHT RIB FRACTURES X 3.'
print("Original text:", text)
print("Clean text:", clean_narrative(text))
print()

# Use all CPU cores
with mp.Pool(mp.cpu_count()) as pool:
    data['Narrative'] = pool.map(clean_narrative, data['Narrative'])
    


Original text: 72 YOF SLIPPED AND FELL ON THE FLOOR THIS AM. DX: L-3, L-4 FRACTURE, RIGHT RIB FRACTURES X 3.
Clean text: Patient slipped and fell on the floor this morning. Clinical diagnosis: l-3, l-4 fracture, right rib fractures x 3.



In [11]:
data['Narrative']=data['Narrative'].str.upper()
data[['Narrative']]

,Narrative
0,PATIENT REPORTS HE FELL 1 WEEK AND COMPLAINS O...
1,A PATIENT BENT TO PICK UP CRATE AT HOME TO ED ...
2,PATIENTRIDING ON MOUNTAIN BIKE PRACTICING FELL...
3,14 MONTH OLD FEMALE ABRASION FOR NOSE AND FORE...
4,4YR M PLAYING WITH TOY KITCHEN APPLIANCE AND ...
...,...
242139,PATIENT FOOT PAIN SP FALL OFF BIKE TODAY
242140,PATIENT LAC HEAD ON HAMOMOCK HIT HEAD ON FENC...
242141,PATIENT FROM THE NURSING HOME FELL OUT OF A WH...
242142,PATIENT CO PAIN TO BOTTOM OF HER FEET 22 CALLU...


In [12]:
replace_list=['ANKLE', 'ARM', 'BODY_PART', 'CHEST', 'CONTUSION', 'CUT', 'EAR', 'ELBOW', 'EYE', 'FACE', 'FINGER', 'FOOT', 'FOREHEAD', 'FRACTURE', 'FX', 'HAND', 'HEAD', 'HIP', 'KNEE', 'LAC', 'LACERATION', 'LEG', 'LOC', 'LOSE', 'NECK', 'PAIN', 'SHOULDER', 
'SPRAIN', 'STRAIN', 'SWELL', 'THUMB', 'TOE', 'WRIST','ABRASION', 'ACHE', 'BREAK', 'BURN', 'CHIN', 'CUT', 'ER', 'FRACTURE', 'FX', 'HIT', 'INJURY', 'LACERATION', 'LIP', 'LOSE', 'LOC', 'MOUTH', 'NOSE', 'PAIN', 'RIB', 'SCALP', 'SPRAIN', 'STRAIN', 'SWELL', 'TOE', 'TWIST', 'WRIST']

for i in replace_list:
  data['Narrative'] = data['Narrative'].str.replace(i, '')
  data['Narrative_LLM']= data['Narrative_LLM'].str.replace(i, '')


data['Narrative'] = data['Narrative'].str.replace('YOM', '')
data['Narrative'] = data['Narrative'].str.replace('YOF', '')
data['Narrative'] = data['Narrative'].str.replace('YR', '')
data['Narrative'] = data['Narrative'].str.replace('OLD', '')
data['Narrative'] = data['Narrative'].str.replace('MALE', '')
data['Narrative'] = data['Narrative'].str.replace('FEMALE', '')
data['Narrative'] = data['Narrative'].str.replace(' YO ', '')
data['Narrative'] = data['Narrative'].str.replace('YO ', '')
data['Narrative'] = data['Narrative'].str.replace(' F ', '')
data['Narrative'] = data['Narrative'].str.replace('YF', '')
data['Narrative'] = data['Narrative'].str.replace(' M ', '')
data['Narrative'] = data['Narrative'].str.replace('ACCIDENTALLY','')
data['Narrative'] = data['Narrative'].str.replace('AGO', '')
data['Narrative'] = data['Narrative'].str.replace('TODAY', '')
data['Narrative'] = data['Narrative'].str.replace('YESTERDAY', '')
data['Narrative'] = data['Narrative'].str.replace('PATIENT', '')
data['Narrative'] = data['Narrative'].str.replace(' PT ', '')
data['Narrative'] = data['Narrative'].str.replace('INJURY', '')
data['Narrative'] = data['Narrative'].str.replace('REPORT', '')
data['Narrative'] = data['Narrative'].str.replace('HURT', '')
data['Narrative'] = data['Narrative'].str.replace('INJ', '')
data['Narrative'] = data['Narrative'].str.replace('FELL', 'FALL')
data['Narrative'] = data['Narrative'].str.replace('INJURE', '')
data['Narrative'] = data['Narrative'].str.replace('JURED', '')
data['Narrative'] = data['Narrative'].str.replace('URED', '')
data['Narrative'] = data['Narrative'].str.replace(' ED', '')
data['Narrative'] = data['Narrative'].str.replace(' RT ', '')
data['Narrative'] = data['Narrative'].str.replace(' LT ', '')

data['Narrative_LLM'] = data['Narrative_LLM'].str.replace('fell', 'fall')


In [15]:
data_core=data.copy() #[core_col]

In [13]:
212347-21000

191347

In [16]:
# please change these hard coded numbers accordingly
data_core=data_core.sample(frac=1,random_state=42).reset_index(drop=True)
data_test=data_core.head(21000).reset_index(drop=True)
data_fit=data_core.tail(191347).reset_index(drop=True)

In [17]:
df_bad=data_fit[data_fit['Disposition_recode_2']==1]
df_good=data_fit[data_fit['Disposition_recode_2']==0]
data_good_sample=df_good.sample(frac=0.2,random_state=42).reset_index(drop=True)
data_bad_sample=df_bad.sample(frac=0.8,random_state=42).reset_index(drop=True)
data_core_sample=pd.concat([data_good_sample,data_bad_sample]).reset_index(drop=True)
data_core_sample=data_core_sample.sample(frac=1,random_state=42).reset_index(drop=True)
data_core_sample.head()

In [20]:
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')


corpus = data_core_sample['Narrative'].fillna('')

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalpha() and token not in stopwords.words('english')]
    tagged_tokens = nltk.pos_tag(tokens)
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token, pos='v') for token, pos in tagged_tokens]
    return ' '.join(lemmatized_tokens)

data_core_sample['Processed_Narrative'] = corpus.apply(preprocess_text)

# Create a TfidfVectorizer object
vectorizer = TfidfVectorizer(max_features=70, stop_words='english')

# Fit and transform the processed text
tfidf_matrix = vectorizer.fit_transform(data_core_sample['Processed_Narrative'])

with open('tfidf_vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)

# Convert the TF-IDF matrix to a DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())



[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [21]:
corpus_LLM = data_core_sample['Narrative_LLM'].fillna('')

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalpha() and token not in stopwords.words('english')]
    tagged_tokens = nltk.pos_tag(tokens)
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token, pos='v') for token, pos in tagged_tokens]
    return ' '.join(lemmatized_tokens)

data_core_sample['Processed_Narrative_LLM'] = corpus_LLM.apply(preprocess_text)

# Create a TfidfVectorizer object
vectorizer_LLM = TfidfVectorizer(max_features=70, stop_words='english')



# Fit and transform the processed text
tfidf_matrix_LLM = vectorizer_LLM.fit_transform(data_core_sample['Processed_Narrative_LLM'])

with open('tfidf_vectorizer_LLM.pkl', 'wb') as file:
    pickle.dump(vectorizer_LLM, file)

# Convert the TF-IDF matrix to a DataFrame
tfidf_df_LLM = pd.DataFrame(tfidf_matrix_LLM.toarray(), columns=vectorizer_LLM.get_feature_names_out())
tfidf_df_LLM=tfidf_df_LLM.add_suffix('_LLM')

# Concatenate the TF-IDF features with your existing data
data_ready = pd.concat([data_core_sample, tfidf_df,tfidf_df_LLM], axis=1)
#data_ready = pd.concat([data_core_sample, tfidf_df_LLM], axis=1)

In [22]:
new_data=data_test.copy()
corpus_2 = new_data['Narrative'].fillna('')
new_data['Processed_Narrative'] = corpus_2.apply(preprocess_text)
tfidf_new = vectorizer.transform(new_data['Processed_Narrative'])
tfidf_df_new = pd.DataFrame(tfidf_new.toarray(), columns=vectorizer.get_feature_names_out())

corpus_LLM_2 = new_data['Narrative_LLM'].fillna('')
new_data['Processed_Narrative_LLM'] = corpus_LLM_2.apply(preprocess_text)
tfidf_LLM_new = vectorizer_LLM.transform(new_data['Processed_Narrative_LLM'])
tfidf_df_new_LLM = pd.DataFrame(tfidf_LLM_new.toarray(), columns=vectorizer_LLM.get_feature_names_out())
tfidf_df_new_LLM=tfidf_df_new_LLM.add_suffix('_LLM')

data_ready_test = pd.concat([data_test, tfidf_df_new,tfidf_df_new_LLM], axis=1)
#data_ready_test = pd.concat([data_test, tfidf_df_new_LLM], axis=1)

In [3]:
drop_list=['Narrative','Unnamed: 0',
 'CPSC_Case_Number',
 'Treatment_Date','Race',
 'Other_Race',
 'Hispanic','Diagnosis',
 'Other_Diagnosis',
 'Body_Part_2',
 'Diagnosis_2',
 'Other_Diagnosis_2',
 'Disposition','Fire_Involvement','Product_2',
 'Product_3',
 'Alcohol',
 'Drug','Stratum',
 'PSU',
 'Weight',
 'Year',
 'Month',
 'Day',
 'activity_at_injury',
 'injury_mechanism',
 'object_involved',
 '#_prod',
 'Disposition_recode',
 'Disposition_recode_2',
 'body_string',
 'Narrative_LLM',
 'Processed_Narrative',
 'Processed_Narrative_LLM']

drop_list_test=['Narrative','Unnamed: 0',
 'CPSC_Case_Number',
 'Treatment_Date','Race',
 'Other_Race',
 'Hispanic','Diagnosis',
 'Other_Diagnosis',
 'Body_Part_2',
 'Diagnosis_2',
 'Other_Diagnosis_2',
 'Disposition','Fire_Involvement','Product_2',
 'Product_3',
 'Alcohol',
 'Drug','Stratum',
 'PSU',
 'Weight',
 'Year',
 'Month',
 'Day',
 'activity_at_injury',
 'injury_mechanism',
 'object_involved',
 '#_prod',
 'Disposition_recode',
 'Disposition_recode_2',
 'body_string',
 'Narrative_LLM']
 

In [4]:
from sklearn.metrics import f1_score, classification_report, accuracy_score, roc_auc_score
from scipy.stats import mode
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import numpy as np
from sklearn.neural_network import MLPClassifier

# Separate features (X) and target variable (y)
X = data_ready.drop(drop_list, axis=1) 
y = data_ready['Disposition_recode_2']

# Encode target variable if it's categorical
le = LabelEncoder()
y = le.fit_transform(y)

# Split data into training and testing sets
X_train, _, y_train, _ = train_test_split(X, y, test_size=0.2, random_state=42)

X_test=data_ready_test.drop(drop_list_test, axis=1)
y_test=data_ready_test['Disposition_recode_2']
y_test = le.fit_transform(y_test)

# Create a scaler for numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.select_dtypes(include=['number']))

with open('X_scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

X_test_scaled = scaler.transform(X_test.select_dtypes(include=['number']))

# Convert scaled features back to DataFrame
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.select_dtypes(include=['number']).columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.select_dtypes(include=['number']).columns)

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_scaled_df, y_train)

# Combine scaled numerical features with categorical features
X_train_final = X_resampled.copy()
X_test_final = X_test_scaled_df.copy()
y_train_final=y_resampled.copy()


# Define models and their parameter grids for hyperparameter tuning
models = {
    #'KNN': (KNeighborsClassifier(), {'knn__n_neighbors': [3, 5, 7]}),
    #'Random_Forest': (RandomForestClassifier(), {'random_forest__n_estimators': [100, 200], 'random_forest__max_depth': [4, 8]}),
    'XGBoost': (XGBClassifier(objective='binary:logistic', use_label_encoder=False), 
           {'xgboost__learning_rate': [0.1, 0.01, 0.001],            # Step size shrinkage to prevent overfitting
            'xgboost__max_depth': [3, 5, 7],                         # Maximum depth of each tree
            #'xgboost__n_estimators': [100, 200, 300],                # Number of boosting rounds
            #'xgboost__min_child_weight': [1, 3, 5],                  # Minimum sum of instance weight needed in a child
            #'xgboost__subsample': [0.6, 0.8,0.9],                   # Subsample ratio of the training instances
            #'xgboost__colsample_bytree': [0.6, 0.8, 0.9],            # Subsample ratio of columns when constructing each tree
            #'xgboost__gamma': [0.1, 0.5, 1,3],                     # Minimum loss reduction required for a split
            #'xgboost__reg_alpha': [0.01, 0.1,0.5,1],                    # L1 regularization term on weights
            #'xgboost__reg_lambda': [1, 1.5, 2],                      # L2 regularization term on weights
            'xgboost__scale_pos_weight': [1, 5, 10]
            }),
    #'Logistic_Regression': (LogisticRegression(solver='liblinear'), {'logistic_regression__C': [0.1, 1, 10]}),
    #'SVM': (SVC(probability=True), {'svm__C': [0.1, 1, 10], 'svm__kernel': ['linear', 'rbf']}),
    #'NN': (MLPClassifier(max_iter=1000), {'nn__hidden_layer_sizes': [(10,), (50,), (100,)], 'nn__activation': ['relu', 'tanh']}),
}

# Initialize list to store predictions from each model
ensemble_predictions = []
ensemble_probabilities = []

for model_name, (model, param_grid) in models.items():
    print(f"Training {model_name}...")

    # Create a pipeline for preprocessing and model training
    pipeline = Pipeline([
        (model_name.lower(), model)
    ])

    # Perform hyperparameter tuning using GridSearchCV with AUC score
    grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='precision',n_jobs=-1)
    grid_search.fit(X_train_final, y_train_final)

    # Get the best model from the grid search
    best_model = grid_search.best_estimator_

    # Make predictions and predict probabilities on the test set
    y_pred = best_model.predict(X_test_final)
    y_prob = best_model.predict_proba(X_test_final)[:, 1]  # Probability for the positive class
    ensemble_predictions.append(y_pred)
    ensemble_probabilities.append(y_prob)

    # Evaluate the model using accuracy, F1-score, and AUC
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)
    print(f"{model_name} Accuracy: {accuracy}")
    print(f"{model_name} F1 Score: {f1}")
    print(f"{model_name} AUC: {auc}")
    print(classification_report(y_test, y_pred))

# Implement ensemble method (majority voting for predictions, average for probabilities)
ensemble_predictions = np.array(ensemble_predictions)
ensemble_pred_final = mode(ensemble_predictions, axis=0)[0].flatten()

# Average the probabilities for the ensemble AUC
ensemble_probabilities = np.mean(ensemble_probabilities, axis=0)

# Evaluate the ensemble model using accuracy, F1-score, and AUC
ensemble_accuracy = accuracy_score(y_test, ensemble_pred_final)
ensemble_f1 = f1_score(y_test, ensemble_pred_final)
ensemble_auc = roc_auc_score(y_test, ensemble_probabilities)
print("Ensemble Accuracy:", ensemble_accuracy)
print("Ensemble F1 Score:", ensemble_f1)
print("Ensemble AUC:", ensemble_auc)
print(classification_report(y_test, ensemble_pred_final))

with open('xgboost_tfidf_LLM_v5.pkl', 'wb') as file:
    pickle.dump(best_model, file)

Training XGBoost...
XGBoost Accuracy: 0.8281904761904761
XGBoost F1 Score: 0.45366444579043
XGBoost AUC: 0.8577310508236893
              precision    recall  f1-score   support

           0       0.96      0.85      0.90     18778
           1       0.34      0.67      0.45      2222

    accuracy                           0.83     21000
   macro avg       0.65      0.76      0.68     21000
weighted avg       0.89      0.83      0.85     21000



/anaconda/envs/azureml_py38/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [01:12:21] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/anaconda/envs/azureml_py38/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [01:12:21] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/anaconda/envs/azureml_py38/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [01:12:22] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/anaconda/envs/azureml_py38/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [01:12:22] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/anaconda/envs/azureml_py38/lib/python3.10/site-packages/xgboost/core.py:158: UserWa

NameError: name 'np' is not defined

In [32]:
xxx

NameError: name 'xxx' is not defined

In [ ]:
'''
sematic_distance_hf=[]

for i in range(len(data)):

    doc_vecs = angle.encode([data.at[i,'body_string'],data.at[i,'Narrative_LLM']], normalize_embedding=True)
    sematic_distance_hf.append(cosine_similarity(doc_vecs[0], doc_vecs[1]))

from sentence_transformers import SentenceTransformer, util

# Load the Sentence-BERT model
model = SentenceTransformer('paraphrase-MPNet-base-v2')  # This is a lightweight, fast model

#sematic_distance_bert=[]

for i in range(315900,len(data)):
    #print(i)
    sentence1 = data.at[i,'body_string'] #"cut"
    sentence2 = data.at[i,'Narrative_LLM'] #"bike"

    embedding1 = model.encode(sentence1, convert_to_tensor=True)
    embedding2 = model.encode(sentence2, convert_to_tensor=True)

    similarity = util.cos_sim(embedding1, embedding2).item()
    sematic_distance_bert.append(similarity)
'''
'''
import spacy
import requests

# Load the SpaCy model
nlp = spacy.load('en_core_web_md')

sematic_distance=[]

for i in range(len(data)):
    #print(i)
    word1 = data.at[i,'body_string'] #"cut"
    word2 = data.at[i,'Narrative_LLM'] #"bike"

    # SpaCy similarity
    doc1 = nlp(word1)
    doc2 = nlp(word2)
    similarity = doc1.similarity(doc2)
    sematic_distance.append(similarity)
'''

In [ ]:
import spacy
import contextualSpellCheck

nlp = spacy.load('en_core_web_sm')
contextualSpellCheck.add_to_pipe(nlp)
doc = nlp('Income was $9.4 milion compared to the prior year of $2.7 milion.')

In [1]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

sentences = ['That is a happy person', 'That is a very happy person']

model = SentenceTransformer('Alibaba-NLP/gte-large-en-v1.5', trust_remote_code=True)
embeddings = model.encode(sentences)

/anaconda/envs/azureml_py38/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-11-08 15:43:12.205899: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731080592.949434    3382 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731080593.161591    3382 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-08 15:43:15.142342: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/71.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [1]:
pip install -U angle-emb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.2/192.2 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 93.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 11.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.5/317.5 kB 23.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
from angle_emb import AnglE
from angle_emb.utils import cosine_similarity

angle = AnglE.from_pretrained('WhereIsAI/UAE-Large-V1', pooling_strategy='cls').cuda()
doc_vecs = angle.encode([
    'leg',
    'ride bike',
    'eye'
], normalize_embedding=True)

for i, dv1 in enumerate(doc_vecs):
    for dv2 in doc_vecs[i+1:]:
        print(cosine_similarity(dv1, dv2))

The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
2024-11-09 02:46:42.813068: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731120403.505347    3430 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731120403.706745    3430 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-09 02:46:45.675043: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

0.9506208896636963
0.34343990683555603
0.3499281108379364


In [3]:
doc_vecs = angle.encode([
    'leg',
    'swimming in the pool',
    'eye'
], normalize_embedding=True)

for i, dv1 in enumerate(doc_vecs):
    for dv2 in doc_vecs[i+1:]:
        print(cosine_similarity(dv1, dv2))

0.5567008852958679
0.5893767476081848
0.4712052643299103


In [9]:
doc_vecs = angle.encode([
    'leg',
    'play basketball',
    'eye'
], normalize_embedding=True)

cosine_similarity(doc_vecs[0], doc_vecs[1])

In [10]:
print(cosine_similarity(doc_vecs[0], doc_vecs[1]))
print(cosine_similarity(doc_vecs[1], doc_vecs[2]))

0.5214386582374573
0.42240607738494873


In [4]:
doc_vecs

array([[ 0.0037621 , -0.03200618, -0.01649955, ..., -0.02925281,
         0.00287279,  0.00504245],
       [ 0.01475093,  0.00713948, -0.0093455 , ...,  0.00296938,
         0.02412086, -0.00261443],
       [-0.01934158, -0.01809479, -0.02410525, ..., -0.00193071,
        -0.020917  ,  0.01015348]], dtype=float32)